In [16]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
sys.path.insert(0, '/home/kenlee/energy_market_project/scripts/MySQL_scripts/')
from DAM_prices_by_SP import Feature_Processor
import time
Jupyter Notebook
Keras_Testing

Python 2

    File
    Edit
    View
    Insert
    Cell
    Kernel
    Help



In [17]:
lzhub = 'LZ_NORTH'
model = 'A'
fp = Feature_Processor()
fp.query('2012-01-01', '2012-12-31')
feature_df = fp.construct_feature_vector_matrix(lzhub, model)
fp.train_test_validate()
norm_train_set = fp.scale_num_features(lzhub, model, fp.train_df, 'standard_scale', set_type = 'Train')
norm_test_set = fp.scale_num_features(lzhub, model, fp.test_df, 'standard_scale', set_type = 'Test')
print(norm_train_set)
final_train_set = fp.convert_dfs_to_numpy(norm_train_set)
final_test_set = fp.convert_dfs_to_numpy(norm_test_set)
x_train, y_train = final_train_set
x_test, y_test = final_test_set


                     Holiday   P(h-24)  P(h-168)  Day0  Day1  Day2  Day3  \
2012-01-23 21:00:00        0 -0.157678 -0.209416   1.0   0.0   0.0   0.0   
2012-01-23 22:00:00        0 -0.195357 -0.219910   1.0   0.0   0.0   0.0   
2012-01-24 00:00:00        0 -0.233520 -0.218452   0.0   1.0   0.0   0.0   
2012-01-24 01:00:00        0 -0.291247 -0.079992   0.0   1.0   0.0   0.0   
2012-01-24 02:00:00        0 -0.301392 -0.007992   0.0   1.0   0.0   0.0   
2012-01-24 03:00:00        0 -0.300909 -0.110599   0.0   1.0   0.0   0.0   
2012-01-24 04:00:00        0 -0.298493 -0.134210   0.0   1.0   0.0   0.0   
2012-01-24 05:00:00        0 -0.274581 -0.199214   0.0   1.0   0.0   0.0   
2012-01-24 06:00:00        0 -0.198739 -0.256055   0.0   1.0   0.0   0.0   
2012-01-24 07:00:00        0  0.066226 -0.330970   0.0   1.0   0.0   0.0   
2012-01-24 08:00:00        0  0.070815 -0.354873   0.0   1.0   0.0   0.0   
2012-01-24 09:00:00        0 -0.090289 -0.361577   0.0   1.0   0.0   0.0   
2012-01-24 1

In [18]:
model = Sequential()
model.add(Dense(30, init = 'glorot_uniform', activation = 'tanh', input_dim = x_train.shape[1]))
model.add(Dense(1, init = 'zero', activation = 'linear'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics = ['accuracy'])
model.summary()
model.fit(x_train, y_train, nb_epoch=100)



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_3 (Dense)                  (None, 30)            1410        dense_input_2[0][0]              
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 1)             31          dense_3[0][0]                    
Total params: 1441
____________________________________________________________________________________________________
Epoch 1/100
5078/5078 [==============================] - 0s - loss: 0.8966 - acc: 0.0000e+00     
Epoch 2/100
5078/5078 [==============================] - 0s - loss: 0.7491 - acc: 0.0000e+00     
Epoch 3/100
5078/5078 [==============================] - 0s - loss: 0.6699 - acc: 0.0000e+00     
Epoch 4/100
5078/5078 [==============================] - 0s - loss: 0.6231 - acc:

In [20]:
modelA_features = ['P(h-24)', 'P(h-168)']
modelA_features.append(lzhub + '_SPP')
rescaled_test = fp.inverse_standard_scale(norm_test_set, modelA_features)
plt.plot(rescaled_test[:,2], label='actual')
y_pred = model.predict(x_test)
norm_test_set[lzhub + '_SPP'] = y_pred
scaled_pred = fp.inverse_standard_scale(norm_test_set, modelA_features)
plt.plot(scaled_pred[:,2], label='predicted')
plt.legend()
plt.show()

TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''